In [0]:
#importing sparksession
 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
 
spark = SparkSession.builder.appName("Read").getOrCreate()

In [0]:
file = spark.read.format("json").option("inferSchema", "true").option("header", "true").load("/FileStore/FileStore/emp.json")

file.display()

age,id,name
25,1,Alice
30,2,Bob
null,3,Charlie
null,invalid_record,null
45,4,David


In [0]:
myschema = StructType([StructField("id",IntegerType(),True), StructField("_corrupt_record",StringType(),True)]);

In [0]:
file_perimissive = spark.read.format("json").schema(myschema).option("header", "true").option("mode", "PERMISSIVE").load("/FileStore/FileStore/emp.json")

file_perimissive.display()

id,_corrupt_record
1,null
2,null
3,null
null,"{""id"": ""invalid_record""}"
4,null


In [0]:
file_DROPMAL = spark.read.format("json").schema(myschema).option("header", "true").option("mode", "DROPMALFORMED").load("/FileStore/FileStore/emp.json")

file_DROPMAL.display()

id,_corrupt_record
1,null
2,null
3,null
4,null


In [0]:
file_failfast = spark.read.format("json").schema(myschema).option("header", "true").option("mode", "FAILFAST").load("/FileStore/FileStore/emp.json")

file_failfast.display()

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 78.0 failed 1 times, most recent failure: Lost task 0.0 in stage 78.0 (TID 83) (ip-10-172-219-17.us-west-2.compute.internal executor driver): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/FileStore/FileStore/emp.json.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:701)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:670)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$hasNext$1(FileScanRDD.scala:490)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:483)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:82)
	at org.apache.spark.sql.execution.collect.Collector.$anonfun$processFunc$1(Collector.scala:208)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$3(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.$anonfun$runTask$1(ResultTask.scala:75)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:55)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:179)
	at org.apache.spark.scheduler.Task.$anonfun$run$5(Task.scala:142)
	at com.databricks.unity.EmptyHandle$.runWithAndClose(UCSHandle.scala:126)
	at org.apache.spark.scheduler.Task.$anonfun$run$1(Task.scala:142)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.scheduler.Task.run(Task.scala:97)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$13(Executor.scala:904)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1741)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:907)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.spark.util.ExecutorFrameProfiler$.record(ExecutorFrameProfiler.scala:110)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:761)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
Caused by: org.apache.spark.SparkException: Malformed records are detected in record parsing. Parse Mode: FAILFAST. To process malformed records as null result, try setting the option 'mode' as 'PERMISSIVE'.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.malformedRecordsDetectedInRecordParsingError(QueryExecutionErrors.scala:1936)
	at org.apache.spark.sql.catalyst.util.FailureSafeParser.parse(FailureSafeParser.scala:103)
	at org.apache.spark.sql.execution.datasources.json.TextInputJsonDataSource$.$anonfun$readFile$5(JsonDataSource.scala:215)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at org.apache.spark.util.CompletionIterator.hasNext(CompletionIterator.scala:31)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:616)
	... 28 more
Caused by: org.apache.spark.sql.catalyst.util.BadRecordException: org.apache.spark.SparkRuntimeException: [CANNOT_PARSE_JSON_FIELD] Cannot parse the field name 'id' and the value invalid_record of the JSON token type VALUE_STRING to target

In [0]:
bad_data = spark.read.format("json") \
    .schema(myschema) \
    .option("header", "TRUE") \
    .option("badRecordsPath", "/FileStore/FileStore/bad_data") \
    .load("/FileStore/FileStore/emp.json")



bad_data.display()

id,_corrupt_record
1,null
2,null
3,null
4,null


In [0]:
dbutils.fs.ls("/FileStore/FileStore/bad_data/20241126T034437/bad_records")


Out[30]: [FileInfo(path='dbfs:/FileStore/FileStore/bad_data/20241126T034437/bad_records/part-00000-34b940e2-01b4-4ec6-b56d-2b5071b4fa47', name='part-00000-34b940e2-01b4-4ec6-b56d-2b5071b4fa47', size=296, modificationTime=1732592678000)]

In [0]:
bad_data_read = spark.read.format("json").load("/FileStore/FileStore/bad_data/20241126T034437/bad_records")


bad_data_read.display()


path,reason,record
dbfs:/FileStore/FileStore/emp.json,"org.apache.spark.SparkRuntimeException: [CANNOT_PARSE_JSON_FIELD] Cannot parse the field name 'id' and the value invalid_record of the JSON token type VALUE_STRING to target Spark data type ""INT"".","{""id"": ""invalid_record""}"
